# Import

In [1]:
import re
from pprint import pprint
from collections import defaultdict
# from urllib.parse import urlsplit

import pandas as pd

from db.mongo import MyMongo

# DB

In [2]:
# Original Data: from scraper
with MyMongo() as db:
    all_js = db.find('public_website', 'a1_pages')

<--Mongo Connected.
Mongo Connection Closed.-->


## Insert

In [3]:
# without_js_script = []

# for item in url_script:
#     item.pop('jsScript')
#     without_js_script.append(item)

# with MyMongo() as db:
#     db.delete_and_insert('public_website', 'a2_js_files', without_js_script)

# Functions

In [4]:
def remove_comment(script):
    script = re.sub(r'(\/\/.*\n)', '', script)
    script = re.sub(r'(\/\*[\s\S]*?\*\/)', '', script)
    return script


def get_comment(script):
    one_line = re.findall(r'(\/\/.*\n)', script)
    multi_line = re.findall(r'(\/\*[\s\S]*?\*\/)', script)

    # return string
    return '\n'.join(one_line) + '\n' + '\n'.join(multi_line)


def get_active_x(script):
    return re.findall(r'ActiveXObject\((.*?)\)', script)


def get_exe(script):
    return re.findall(r'\/(.*\.exe)[^c]', script)


def extend_active_x(ax_list, script):
    new_list = get_active_x(script)
    if new_list:
        ax_list.extend(new_list)


def extend_exe(exe_list, script):
    new_list = get_exe(script)
    if new_list:
        exe_list.extend(new_list)


# Javascript

## In-Page JS

### Cursor to Dict

In [5]:
in_page_js = defaultdict(dict)

for u in all_js:
    ax_cmt = []
    ax_srt = []
    exe_cmt = []
    exe_srt = []
    for script in u['jsScript']:
        cmt = get_comment(script)
        extend_active_x(ax_cmt, cmt)
        extend_exe(exe_cmt, cmt)

        srt = remove_comment(script)
        extend_active_x(ax_srt, srt)
        extend_exe(exe_srt, srt)

    in_page_js[u['url']]['ax_cmt'] = ax_cmt
    in_page_js[u['url']]['ax_srt'] = ax_srt
    in_page_js[u['url']]['exe_cmt'] = exe_cmt
    in_page_js[u['url']]['exe_srt'] = exe_srt

i = 0
for k, v in in_page_js.items():
    if i == 4:
        break
    i += 1
    print(k, v)

http://lms.khealth.or.kr {'ax_cmt': [], 'ax_srt': [], 'exe_cmt': [], 'exe_srt': []}
http://kmtp.medicalkorea.or.kr {'ax_cmt': [], 'ax_srt': [], 'exe_cmt': [], 'exe_srt': []}
http://www.khealth.or.kr/healthplan {'ax_cmt': [], 'ax_srt': [], 'exe_cmt': [], 'exe_srt': []}
http://haspa.khidi.or.kr/ {'ax_cmt': [], 'ax_srt': [], 'exe_cmt': [], 'exe_srt': []}


### Dict to DF

In [6]:
# df_ax_script = pd.DataFrame(columns=['url', 'activex'], data=df_data)
df_data_0 = []
df_data_1 = []
df_data_2 = []
df_data_3 = []

for url, script in in_page_js.items():
    for name in script['ax_cmt']:
        df_data_0.append((url, name))
    for name in script['ax_srt']:
        df_data_1.append((url, name))
    for name in script['exe_cmt']:
        remove_garbage = name.split('/')[-1]
        df_data_2.append((url, remove_garbage))
    for name in script['exe_srt']:
        remove_garbage = name.split('/')[-1]
        df_data_3.append((url, remove_garbage))


df_ax_cmt = pd.DataFrame(columns=['url', 'ax_cmt'], data=df_data_0)
df_ax_srt = pd.DataFrame(columns=['url', 'ax_srt'], data=df_data_1)
df_exe_cmt = pd.DataFrame(columns=['url', 'exe_cmt'], data=df_data_2)
df_exe_srt = pd.DataFrame(columns=['url', 'exe_srt'], data=df_data_3)

---

In [7]:
# len(df_ax_srt)  # 201개
df_ax_srt.head()

,url,ax_srt
0,http://www.pharm114.or.kr/,"""Microsoft.XMLDOM"""
1,http://mfds.go.kr/index.do,"""Microsoft.XMLDOM"""
2,http://kras.gwd.go.kr/land_info/info/baseInfo/...,"""Microsoft.XMLDOM"""
3,http://kras.incheon.go.kr:80/land_info/info/ba...,"""Microsoft.XMLDOM"""
4,http://kras.jeju.go.kr:80/land_info/info/baseI...,"""Microsoft.XMLDOM"""


### Active X 종류

In [8]:
df_ax_srt.groupby('ax_srt').agg({'ax_srt':'count'})

,ax_srt
ax_srt,
activeXObject[i],7
"""CMSActiveX.IssacWebProCMS_KCMVP.1""",1
"""ENCActiveX.IssacWebSE_KCMVP.1""",1
"""Microsoft.XMLDOM""",23
"""Microsoft.XMLHTTP""",82
"""Msxml2.XMLHTTP""",62
"""Scripting.FileSystemObject""",4
"""WScript.Shell""",1
"""htmlfile""",1


In [9]:
# len(df_exe_srt)  # 17개
df_exe_srt.head()

,url,exe_srt
0,http://www.ongjin.go.kr/open_content/main/envi...,astx_setup.exe
1,http://www.hs-support.go.kr/hss/uat/uia/hssLog...,GPKISecureWebX.exe
2,http://cals.icpa.or.kr,MiPlatform_SetupDeploy320U_200907.exe
3,https://mail.edunavi.kr:443/jxmail/ko/jsp/logi...,JXCealNT.exe
4,https://mail.edunavi.kr:443/jxmail/ko/jsp/logi...,JXCealPlugin.exe


### exe 파일 종류

In [10]:
df_exe_srt.groupby('exe_srt').agg({'exe_srt':'count'})

,exe_srt
exe_srt,
CX60_Plugin_u_setup.exe,1
CX60u_OCX_setup.exe,1
GPKISecureWebX.exe,2
HTML5v1.3.7.exe,1
ISignPlusWA_Setup_3.0.13.1.exe,1
JXCealNT.exe,1
JXCealPlugin.exe,1
KSignCASE For HTML5_Windows_v1.3.13_180820.exe,1
MiPlatform_SetupDeploy320U_200907.exe,1


## External JS File

In [11]:
with MyMongo() as db:
    js_files = db.get_df_from_table('public_website', 'a3_js_files_per_domain')

<--Mongo Connected.
Mongo Connection Closed.-->


In [12]:

# js_files.head()
len(js_files)

130462

In [13]:
def remove_js_file_contain_keyword(df, keyword):
    return df[~df['jsFile'].str.contains(keyword)]

In [14]:
js_files_1 = remove_js_file_contain_keyword(js_files, 'jquery')  # 94142

In [15]:
js_files_1 = remove_js_file_contain_keyword(js_files_1, '.min.') # 89416
print(len(js_files_1))

89416


In [16]:
js_files_1 = remove_js_file_contain_keyword(js_files_1, 'css/') # 88945
print(len(js_files_1))

88945


In [17]:
js_files_1 = remove_js_file_contain_keyword(js_files_1, 'google') # 88307
print(len(js_files_1))

88307


In [18]:
js_files_1 = remove_js_file_contain_keyword(js_files_1, 'naver') # 87705
print(len(js_files_1))

87705


In [19]:
js_files_2 = js_files_1.drop(columns=['_id']); js_files_2

,jsFile,netLoc
1,/resources/js/common.js,www.khealth.or.kr
2,/MagicLine4NP/magicline4np/js/magicline.js?v=2...,www.khealth.or.kr
4,/resources/js/common.ccis.js,www.khealth.or.kr
5,/resources/component/crosseditor/js/namo_scrip...,www.khealth.or.kr
7,/resources/js/plani.scroller_slide.js,www.khealth.or.kr
12,/resources/js/plani.popup_zone.js,www.khealth.or.kr
14,https://use.fontawesome.com/2d530f5474.js,haspa.khidi.or.kr
22,/js/khidi.js,haspa.khidi.or.kr
24,js/global5152.js?ver=1.0,matchup.kr
27,js/navigation5152.js?ver=1.0,matchup.kr


In [20]:
js_files_2

,jsFile,netLoc
1,/resources/js/common.js,www.khealth.or.kr
2,/MagicLine4NP/magicline4np/js/magicline.js?v=2...,www.khealth.or.kr
4,/resources/js/common.ccis.js,www.khealth.or.kr
5,/resources/component/crosseditor/js/namo_scrip...,www.khealth.or.kr
7,/resources/js/plani.scroller_slide.js,www.khealth.or.kr
12,/resources/js/plani.popup_zone.js,www.khealth.or.kr
14,https://use.fontawesome.com/2d530f5474.js,haspa.khidi.or.kr
22,/js/khidi.js,haspa.khidi.or.kr
24,js/global5152.js?ver=1.0,matchup.kr
27,js/navigation5152.js?ver=1.0,matchup.kr


In [21]:
# with MyMongo() as db:
#     db.delete_and_insert_df('public_website', 'a4_js_files_processed', js_files_2)

In [22]:
# keywords = ['wizvera', 'anysign4pc', 'touchen', 'ipinside', 'softcamp', 'nprotect', 'astx', 'ksign', 'initech']
# result = defaultdict(list)

# for item in js:
#     for keyword in keywords:
#         for file in item['jsFile']:
#             if keyword in file.lower():
#                 result[keyword].append(item['url'])
